In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
from sklearn.preprocessing import LabelEncoder

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import *
from keras.preprocessing.sequence import pad_sequences

In [2]:
# Import tweets and sentiment data

dataset = pd.read_excel('sample_tweets.xlsx')
tweets = dataset['SentimentText'].values
sent = dataset['Sentiment'].values

In [3]:
tweets[0]

"#gstnfailed @nsitharaman .. Such a bold step.. Made by extending 3 days.. Now v know.. V should never vote & give majority to @BJP4India @PMOIndia .. Such a foolish step.. Really saddened to know.. Dat u don't understand d stress. of GST professionals.."

In [32]:
# Create emedding_vector for each word from Glove

f = open("glove.6B.50d.txt", encoding="utf-8")
glove_emb = {}
for line in f:
    arr = line.split()
    word = arr[0]
    vect = np.asarray(arr[1:], dtype="float")
    glove_emb[word] = vect

f.close()
print(glove_emb['car'])

[ 0.47685  -0.084552  1.4641    0.047017  0.14686   0.5082   -1.2228
 -0.22607   0.19306  -0.29756   0.20599  -0.71284  -1.6288    0.17096
  0.74797  -0.061943 -0.65766   1.3786   -0.68043  -1.7551    0.58319
  0.25157  -1.2114    0.81343   0.094825 -1.6819   -0.64498   0.6322
  1.1211    0.16112   2.5379    0.24852  -0.26816   0.32818   1.2916
  0.23548   0.61465  -0.1344   -0.13237   0.27398  -0.11821   0.1354
  0.074306 -0.61951   0.45472  -0.30318  -0.21883  -0.56054   1.1177
 -0.36595 ]


In [33]:
print(myTokenizer(tweets[0]))

['gstnfailed', 'nsitharaman', 'bold', 'step', 'made', 'extending', 'days', 'know', 'never', 'vote', 'give', 'majority', 'bjp', 'india', 'pmoindia', 'foolish', 'step', 'really', 'saddened', 'know', 'dat', 'understand', 'stress', 'gst', 'professionals']


In [34]:
re = RegexpTokenizer("[a-zA-Z]+")
sw = stopwords.words("english")

In [35]:
unique_vocab = {}

weight = []
k=1;
X = tweets
X_train = np.zeros((10000, 50))

for i in range(X.shape[0]):
    words = re.tokenize(X[i])
    words = [w for w in words if w not in sw]
    
    for j in range(min(len(words), 50)):
        if words[j].lower() in unique_vocab:
            X_train[i][j] = unique_vocab[words[j].lower()]
            
        elif words[j] in glove_emb:
            weight.append(glove_emb[words[j]])
            X_train[i][j] = k
            unique_vocab[words[j].lower()] = k
            k += 1

In [36]:
X_train[0]

array([ 0.,  0.,  0.,  1.,  2.,  0.,  3.,  4.,  0.,  5.,  6.,  5.,  7.,
        8.,  9., 10.,  0.,  0.,  0.,  0., 11.,  2.,  0., 12.,  6.,  0.,
       13., 14., 15.,  0., 16.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [37]:
wt_mat = [np.zeros((50,))] + weight
wt_mat = np.array(wt_mat)

In [38]:
print(wt_mat.shape)

(4377, 50)


In [39]:
le=LabelEncoder()
y_enc = le.fit_transform(sent)

Y = to_categorical(y_enc)

In [40]:
print(X_train[1], Y[1])

[ 0.  0.  0.  0.  0. 17. 18. 19. 20.  0.  0.  0.  0. 21.  0.  0.  0. 22.
 23. 18. 24. 25. 26.  0. 27.  0. 28.  0.  0. 29. 30. 31. 32. 33.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.] [0. 1. 0.]


In [41]:
# Create LSTM model

model = Sequential()
model.add(Embedding(4377, 50, mask_zero=True, weights=[wt_mat], input_length=50, trainable=False))
model.add(LSTM(32, return_sequences=True, activation='relu'))
model.add(Dropout(0.25))
model.add(LSTM(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            218850    
_________________________________________________________________
lstm (LSTM)                  (None, 50, 32)            10624     
_________________________________________________________________
dropout (Dropout)            (None, 50, 32)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 3)                 99        
Total params: 237,893
Trainable params: 19,043
Non-trainable params: 218,850
_____________________________________________

In [ ]:
# Train model

hist = model.fit(X_train, Y, validation_split=0.2, epochs=10, batch_size=32)

In [ ]:
plt.plot(hist.history['acc'], r='b')
plt.plot(hist.history['val_acc'], r='g')
plt.show()

# Make predictions

for ix in range(5):
    p = model.predict(X[ix])
    print(X[ix], le.inverse_transform(p))